# JDBC

Java DataBase Connectivity Standard 

standardizes the Java API that a database product must implement s.t. that a Java program can 
- open a connection
- send SQL queries
- recieve returned tupples as Java objects conforming to a certain API s.t. that they can be post processed w/ Java code

Not a new concept: Launching SQL inquiries from host lang and post processing those results was first done in C++
- ODBC (Microsoft standard, Object DB Connectivity)
- Another obscure standard. However both are outdated and were more mentioned as a history lesson

## Three-Tier Architecture

![](images/2022-10-25-21-41-52.png)



CSE 132B: DB applications (sp 23)
- focuses on the upper 2 tiers of this arch
- building websites built on top of DB server

CSE 132A: curr
- focuses on the bottom tier of this arch

![](images/2022-11-10-09-24-17.png)


![](images/2022-11-10-09-35-26.png)

JDBC driver provided by the vendor implementing SQL standards.
- either interacts directly w/ DB server
- or interacts w/ DB middleware that would interact w/ DB server

> EX use case of middleware: Could be useful in integrating tables that come from multiple servers, gives the impression of a centralized database server

JDBC EX

```java
// import JDBC
import java.sql.*;

class JdbcTest {
    public static void main (String args[]) throws SQLException, ClassNotFoundException {
        // Load PostgreSQL driver
        Class.forName("org.postgresql.Driver");\

        // Connect to the local database
        Connection conn = DriverManager.getConnection(
            "djbc:postgresql://hostname:port/dbname",
            "username",
            "password"
        )

        // Execute query asking for student names
        Statement stmt = conn.createStatement();
        ResultSet rset = stmt.executeQuery("SELECT age, name FROM Student");

        // Print the name out (name is the 2nd attribute of Student)
        while (rset.next())
            System.out.println(rset.getString(2));

        // Close the result set, statement, and the connection
        rset.close();
        stmt.close();
        conn.close();
    }
}
```

## SQL Injection

`executeQuery` method has problems due to lack of proecssing/cleaning the passed in input string:

1. Might not be valid SQL and cause crashes/errors
2. SQL Injection Atk. Malicious string will cause DB system to execute instructions that we might not want the user to be able to execute

Example:

```java
// ssn = whatever user typed into the user interface

String query = "SELECT FROM student s
                WHERE s.ssn = " + ssn + ";";
stmt.executeQuery(query)
```

What if user typed in `"123456; DELETE FROM students"`?

```java
String query = "SELECT FROM student s
                WHERE s.ssn = " + "123456; DELETE FROM students" + ";"; 
```

Which would cause the server to execute the following:
```sql
SELECT FROM student s
WHERE s.ssn = 123456;
DELETE FROM students
```

Malicious user can sneak in extra code to do stuff like delete information, leak information, etc.

## PreparedStatement Object

If you want to execute a STateent object many times, it will normally reduce execution time to use a PreparedStatement object instead (in addition to guarding against SQL Injection)

```java
// Create PreparedStatement
// '?' contains parameters
PreparedStatement updateStud = 
    conn.prepareStatement("UPDATE Student SET name = ?
    WHERE lastname LIKE ?");

// Instantiate parameters and execute the PreparedStatement
updateStud.setString(1, "John");
updateStud.setString(2, "Smith");

updateStud.executeUpdate();
```

Use case: a lot of users interacting w/ website.

Sending just args to the db is computationally cheaper than sending entire queries: less parsing, compiling, optimizing the query

EX: these 2 do the same thing (assuming non malicious user)

```java
String updateString = "UPDATE COFFEES SET SALES = 75" + "WHERE COF_NAME LIKE 'Colombian'";
stmtm.executeUpdate(updateString)
```

```java
PreparedStatement updateSales = con.prepareStatement("UPDATE COFFEES SET SALES = ? WHERE COF_NAME LIKE ?");
updateSales.setInt(1, 75);
updateSales.setString(2, "Colombian");
updateSales.executeUpdate();
```

### Retrieving values from a ResultSet

> Recall that Java has no knowledge of your DB schema, so improper queries will only get punished at runtime

Retrieves the values of the designated column in the current row of this ResultSet object as an int in Java.

- `int getInt(int columnIndex)`
- `int getInt(String columnName)`

Retrieves the value of the designated column in the current row of this ResultSet object as a string in Java.

- `String getString(int columnIndex)`
- `String getString(String columnName)`

### Using Transactions

When a connection is created, it is in auto-commit mode. This means that each individual SQL statment is treated as a transaction and will be automatically committed right after it is executed. To create transactions, do the following:

```java
conn.setAutoCommit(false);
// ...
// transaction here
// ...
conn.commit();
conn.setAutoCommit(true);
```

Basically, controlling the atomicity of SQL statments. Use case? Transferring money from checkings to savings. Any error in between a non-transactional set of queries might result in deductions from checkings without updating the credits into savings.

### Catching Exceptions

JDBC lets you see the warnings and exceptions generated by you DMBS and by the java compiler. To see exceptions, you can have a catch block print them out:

```java
try {
    // Code that could gen exception here
} catch(SQLException ex) {
    // Catch block to print out info about caught exceptions
    System.err.println("SQLException: " + ex.getMessage());
}
```